In [161]:
import json
import base64
import os
import io

from models import ToolWeapon, ItemEffectsInfo, AttacksInfo
from IPython import display
from PIL import Image
from pathlib import Path

In [162]:
def get_icon_image(icon_path: Path, modicon_path: Path, size: int = 32) -> Image:
    icon = None
    if os.path.exists(icon_path) and os.path.isfile(icon_path):
        icon = Image.open(icon_path)
        if icon.mode != 'RGBA':
            icon = icon.convert('RGBA')
        icon = icon.resize((size, size), Image.LANCZOS)

        if os.path.exists(modicon_path) and os.path.isfile(modicon_path):
            modifier = Image.open(modicon_path)
            if modifier.mode != 'RGBA':
                modifier = modifier.convert('RGBA')
            modifier = modifier.resize((42, 42))

            try:
                icon.paste(modifier, (0, 0), modifier)
            except:
                return icon

    return icon

def image_to_base64(image: Image) -> str:
    if image is None:
        return ''
    buffer = io.BytesIO()
    image.save(buffer, format='PNG')
    return base64.b64encode(buffer.getvalue()).decode()

In [163]:
CSS_MATERIAL = """
<head>
    <link href="https://fonts.googleapis.com/css2?family=Poppins:ital,wght@0,100;0,200;0,300;0,400;0,500;0,600;0,700;0,800;0,900;1,100;1,200;1,300;1,400;1,500;1,600;1,700;1,800;1,900&display=swap" rel="stylesheet">
    <style>
        * {
            font-family: 'Poppins', sans-serif;
        }

        body {
            font-family: 'Poppins', sans-serif;
            background-color: #403f57;
            height: 100%;
        }
        .row {
            display: flex;
            justify-content: space-between;
        }
        .col {
            flex: 1;
        }
        .p-0 {
            padding: 0;
        }
        .p-1 {
            padding: 5px;
        }
        .p-2 {
            padding: 10px;
        }
        .px-0 {
            padding-left: 0;
            padding-right: 0;
        }
        .py-0 {
            padding-top: 0;
            padding-bottom: 0;
        }
        .px-1 {
            padding-left: 5px;
            padding-right: 5px;
        }
        .py-1 {
            padding-top: 5px;
            padding-bottom: 5px;
        }
        .px-2 {
            padding-left: 10px;
            padding-right: 10px;
        }
        .py-2 {
            padding-top: 10px;
            padding-bottom: 10px;
        }
        .px-3 {
            padding-left: 15px;
            padding-right: 15px;
        }
        .py-3 {
            padding-top: 15px;
            padding-bottom: 15px;
        }
        .px-4 {
            padding-left: 20px;
            padding-right: 20px;
        }
        .py-4 {
            padding-top: 20px;
            padding-bottom: 20px;
        }
        .m-0 {
            margin: 0;
        }
        .m-1 {
            margin: 5px;
        }
        .w-100 {
            width: 100%;
        }
        .h-100 {
            height: 100%;
        }
        .center {
            align-content: center;
            text-align: center;
        }
        .card {
            background-color: #1e1d39;
            color: #e6e6e6;
            padding: 10px 25px 10px 25px;
            border-radius: 5px;
            border: 1px solid #808080;
        }
        .title {
            font-size: 1.25em;
            font-weight: 500;
        }
        .subtitle {
            font-size: 0.75em;
            font-weight: 400;
            color: #b3b3b3;
        }
        .inline {
            display: inline;
        }
        .effects > table {
            width: 100%; /* Ajusta a tabela para preencher o contêiner .effects */
            border-collapse: collapse; /* Remove o espaçamento padrão entre bordas de células */
            margin-bottom: 16px; /* Espaçamento abaixo da tabela */
        }
        .effects > table th {
            background-color: #468847; /* Cor de fundo do cabeçalho */
            color: white; /* Cor do texto do cabeçalho */
            padding: 8px; /* Espaçamento interno do cabeçalho */
            border: 1px solid #a2a9b1; /* Bordas do cabeçalho */
            border-radius: 2px; /* Raio da borda do cabeçalho */
        }
        .effects > table td {
            padding: 8px; /* Espaçamento interno das células */
            border: 1px solid #a2a9b1; /* Bordas das células */
        }
        .effects > table tr {
            border: 1px solid #a2a9b1; /* Bordas das linhas */
        }
        /* Opção para aplicar bordas arredondadas apenas nas células de canto do cabeçalho */
        .effects > table tr:first-child th:first-child {
            border-top-left-radius: 2px;
        }
        .effects > table tr:first-child th:last-child {
            border-top-right-radius: 2px;
        }
        .icon-text {
            list-style-type: none; /* Remove o marcador padrão do li */
            margin: 10px; /* Espaçamento opcional */
        }
        .icon-text img {
            vertical-align: middle; /* Alinha o ícone verticalmente ao meio em relação ao texto */
            margin-right: 5px; /* Espaçamento entre o ícone e o texto */
        }
        .tools-weapons {
            grid-gap: 10px;
            display: grid;
            grid-template-columns: repeat(auto-fill, minmax(200px, calc(100% / 2 - (5px * 1))));
        }
        .form-field {
            position: relative;
            margin: 20px 0;
            padding: 10px 0 0;
        }

        .input-material {
            font-size: 16px;
            width: 100%;
            padding: 11px;
            display: block;
            border: 1px solid #808080;
            border-radius: 5px;
            background-color: #1e1d39;
        }

        .input-material:focus {
            outline: none;
            padding: 10px;
            border: 2px solid #468847 /* Primary color */
        }

        .label-material {
            position: absolute;
            left: 0;
            top: 10px;
            padding: 12px 0px 12px 14px;
            color: #999;
            font-size: 16px;
            transition: all 0.3s ease;
            pointer-events: none;
            z-index: 2;
        }

        .label-box {
            display: none;
            position: absolute;
            left: 0;
            top: 10px;
            color: #1e1d39;
            height: 2px;
            width: 100%;
        }

        .input-material:focus ~ .label-material,
        .input-material:not(:placeholder-shown) ~ .label-material {
            top: -11px;
            left: 0;
            font-size: 14px;
            font-weight: 600;
            color: #468847;
            transition: all 0.3s ease;
        }

        .input-material:focus ~ .label-material>.label-box,
        .input-material:not(:placeholder-shown) ~ .label-material>.label-box {
            display: block;
            background: #1e1d39;
            top: 21px;
            margin: 0px 0px 0px 8px;
            z-index: -1;
        }
    </style>
</head>
"""

In [164]:
def generate_search_field() -> str:
    return """
    <div class="row form-field">
        <input type="text" id="search" class="w-100 p-1 input-material" placeholder="">
        <label for="search" class="label-material">Search for Weapon / Tool ...<div for="search" class="label-box"></div></label>
    </div>
    <script>
        document.getElementById('search').addEventListener('input', function() {
            var search = this.value.toLowerCase();
            var cards = document.getElementsByClassName('card');
            for (var i = 0; i < cards.length; i++) {
                var card = cards[i];
                var title = card.getElementsByClassName('title')[0].innerText.toLowerCase();
                if (title.includes(search)) {
                    card.style.display = '';
                } else {
                    card.style.display = 'none';
                }
            }
        });
    </script>
    """

In [165]:
def generate_effects_html(effects_info: ItemEffectsInfo) -> str:
    if effects_info.count_valid_effects() == 0:
        return ''

    html = '<h4 style="margin-bottom: 0px; padding-bottom: 0px;">Effects</h4>'
    html += '<ul style="margin-top: 0; padding-top: 0px;">'
    for effect in effects_info.main_status_effects:
        if effect.display_name.text == 'UNKNOWN':
            continue
        description = f' - {effect.description.text}' if effect.description.text != 'UNKNOWN' else ''

        icon = get_icon_image(effect.icon_path, '', 28)
        icon_base64 = image_to_base64(icon)
        html += f'<li class="icon-text"><img src="data:image/png;base64,{icon_base64}">'
        html += f'<span style="vertical-align: middle; font-weight: 500;"> {effect.display_name.text}</span>'
        html += f'<span style="vertical-align: middle;"> {description}</span>'
        html += '</li>'
    for effect in effects_info.hidden_status_effects:
        if effect.display_name.text == 'UNKNOWN':
            continue
        description = f' - {effect.description.text}' if effect.description.text != 'UNKNOWN' else ''

        icon = get_icon_image(effect.icon_path, '', 28)
        icon_base64 = image_to_base64(icon)
        html += f'<li class="icon-text"><img src="data:image/png;base64,{icon_base64}">'
        html += f'<span style="vertical-align: middle; font-weight: 500;"> {effect.display_name.text}</span>'
        html += f'<span style="vertical-align: middle;"> {description}</span>'
        html += '</li>'
    html += '</ul>'

    return html

In [166]:
def generate_melee_attacks_html(attacks_info: AttacksInfo) -> str:
    html = '<h4 style="margin-bottom: 0px; padding-bottom: 0px;">Attacks</h4>'
    html += '<table>'
    html += '<thead>'
    html += '<tr>'
    html += '<th>Attack</th>'
    html += '<th>Damage</th>'
    html += '<th>Stun</th>'
    html += '<th>Stamina</th>'
    html += '</tr>'
    html += '</thead>'
    html += '<tbody>'
    for index, attack in enumerate(attacks_info.main_combo):
        html += '<tr>'
        html += f'<td>Attack {index + 1}</td>'
        html += f'<td>{attack.main_damage_data.damage}</td>'
        html += '</tr>'
    html += '</tbody>'
    html += '</table>'

    return html

In [167]:
def generate_card_html(tool_weapon: ToolWeapon) -> str:
    subtitle = f'Rarity: {tool_weapon.rarity_tag.split(".")[-1]}'
    subtitle += f' | Tier {tool_weapon.tier}'

    icon = get_icon_image(tool_weapon.icon_path, tool_weapon.icon_modifier_path, 128)
    icon = image_to_base64(icon)

    return f"""
    <div class="card">
        <div class="row h-100">
            <div class="col">
                <div class="title">{tool_weapon.display_name.text}</div>
                <div class="subtitle">{subtitle}</div>
                {tool_weapon.description.text}
                {generate_effects_html(tool_weapon.item_effects_info)}
                {generate_melee_attacks_html(tool_weapon.melee_attacks_info)}
            </div>
            <div class="px-2 px-1 center" style="width: 128px">
                <img src="data:image/jpeg;base64,{icon}">
            </div>
        </div>
    </div>
    """

In [168]:
def generate_complete_html(tool_weapons):
    cards = ''.join(generate_card_html(tool_weapon) for tool_weapon in tool_weapons)
    return f"""
<!DOCTYPE html>
<html>
    {CSS_MATERIAL}
    <body style="padding: 20px;">
        {generate_search_field()}
        <div class="tools-weapons">
            {cards}
        </div>
    </body>
</html>
    """

In [169]:
tool_weapons_path = './data/crawled/1.4/tools_weapons.json'
tool_weapons = json.load(open(tool_weapons_path))

for weapon_name, weapon_data in tool_weapons.items():
    tool_weapons[weapon_name] = ToolWeapon.from_dict(weapon_data)

In [170]:
with open('./index.html', 'w') as f:
    f.write(generate_complete_html(list(tool_weapons.values())))

In [1]:
hex(0x20+0x28)

'0x48'